In [2]:
!pip install -q transformers datasets evaluate accelerate


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
import torch


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
dataset = load_dataset("imdb")
dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
small_train = dataset["train"].shuffle(seed=42).select(range(4000))   # 4k
small_test  = dataset["test"].shuffle(seed=42).select(range(1000))    # 1k


In [7]:
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2  # positive / negative
).to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",   # pad to fixed length
        truncation=True,
        max_length=256
    )


In [9]:
tokenized_train = small_train.map(tokenize_function, batched=True)
tokenized_test  = small_test.map(tokenize_function, batched=True)


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_test  = tokenized_test.remove_columns(["text"])

tokenized_train = tokenized_train.with_format("torch")
tokenized_test  = tokenized_test.with_format("torch")

tokenized_train[0]


{'label': tensor(1),
 'input_ids': tensor([  101,  2045,  2003,  2053,  7189,  2012,  2035,  2090,  3481,  3771,
          1998,  6337,  2099,  2021,  1996,  2755,  2008,  2119,  2024,  2610,
          2186,  2055,  6355,  6997,  1012,  6337,  2099,  3504, 15594,  2100,
          1010,  3481,  3771,  3504,  4438,  1012,  6337,  2099, 14811,  2024,
          3243,  3722,  1012,  3481,  3771,  1005,  1055,  5436,  2024,  2521,
          2062,  8552,  1012,  1012,  1012,  3481,  3771,  3504,  2062,  2066,
          3539,  8343,  1010,  2065,  2057,  2031,  2000,  3962, 12319,  1012,
          1012,  1012,  1996,  2364,  2839,  2003,  5410,  1998,  6881,  2080,
          1010,  2021,  2031,  1000, 17936,  6767,  7054,  3401,  1000,  1012,
          2111,  2066,  2000, 12826,  1010,  2000,  3648,  1010,  2000, 16157,
          1012,  2129,  2055,  2074,  9107,  1029,  6057,  2518,  2205,  1010,
          2111,  3015,  3481,  3771,  3504,  2137,  2021,  1010,  2006,  1996,
          2060,  2

In [11]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)


In [19]:
batch_size = 8

training_args = TrainingArguments(
    output_dir="bert-imdb-sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    report_to="none",
)



In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipython-input-4103087747.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.352100,0.458999,0.824000
2,0.218700,0.419501,0.882000


TrainOutput(global_step=1000, training_loss=0.31575673961639406, metrics={'train_runtime': 441.1895, 'train_samples_per_second': 18.133, 'train_steps_per_second': 2.267, 'total_flos': 1052444221440000.0, 'train_loss': 0.31575673961639406, 'epoch': 2.0})

In [21]:
metrics = trainer.evaluate()
metrics


{'eval_loss': 0.41950133442878723,
 'eval_accuracy': 0.882,
 'eval_runtime': 15.4571,
 'eval_samples_per_second': 64.695,
 'eval_steps_per_second': 8.087,
 'epoch': 2.0}

In [22]:
label_names = ["negative", "positive"]

def predict_sentiment(text):
    # Tokenize
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    ).to(device)

    # Disable grad for inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        pred_label_id = torch.argmax(probs, dim=-1).item()
        confidence = probs[0, pred_label_id].item()

    return label_names[pred_label_id], confidence


In [26]:
texts = [
    "I absolutely loved this movie, it was fantastic!",
    "This was the worst film I have ever seen.",
    "It was okay, not great but not terrible either."
]

for t in texts:
    label, conf = predict_sentiment(t)
    print(f"Text: {t}\nPredicted: {label} (confidence: {conf:.3f})\n")


Text: I absolutely loved this movie, it was fantastic!
Predicted: positive (confidence: 0.997)

Text: This was the worst film I have ever seen.
Predicted: negative (confidence: 0.997)

Text: It was okay, not great but not terrible either.
Predicted: negative (confidence: 0.947)



In [27]:
!mkdir small_model
!cp bert-imdb-sentiment/checkpoint-*/config.json small_model/
!cp bert-imdb-sentiment/checkpoint-*/tokenizer.json small_model/
!cp bert-imdb-sentiment/checkpoint-*/vocab.txt small_model/


cp: will not overwrite just-created 'small_model/config.json' with 'bert-imdb-sentiment/checkpoint-500/config.json'
cp: will not overwrite just-created 'small_model/tokenizer.json' with 'bert-imdb-sentiment/checkpoint-500/tokenizer.json'
cp: will not overwrite just-created 'small_model/vocab.txt' with 'bert-imdb-sentiment/checkpoint-500/vocab.txt'


In [28]:
!zip -r small_model.zip small_model
from google.colab import files
files.download("BERT-imdb.zip")


  adding: small_model/ (stored 0%)
  adding: small_model/vocab.txt (deflated 53%)
  adding: small_model/tokenizer.json (deflated 71%)
  adding: small_model/config.json (deflated 49%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>